In [ ]:
import numpy as np

import normflows as nf
from normflows import nets
from normflows.flows import Planar, Radial, MaskedAffineFlow, BatchNorm

import uproot
from matplotlib import pyplot as plt

from tqdm.notebook import tqdm
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

import corner

import typing

import anomalydetector
from anomalydetector.models.VAE import NFVAE, VAE
from anomalydetector.models.NICE import NICEModel, NICE_gaussian_loss
from anomalydetector.processing import InMemoryND280EventDataset, nd280EventDataset


if torch.cuda.is_available():
    print("Found cuda device, will use GPU")
else:
    print("No GPU :(")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
with uproot.open('/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run2air_v1.root') as file:

    sample_sum = file['sample_sum'].arrays(
        filter_branch=lambda b: b.name.find("Graph") == -1 and b.typename.find("std::vector") == -1,
        library='pd'
    )

    print (":::::: Available branches ::::::::")

    for branch in file['sample_sum'].branches:
        print(f'  - {branch.name}: {branch.typename}')


In [ ]:

train_ds = InMemoryND280EventDataset(
    #root="/home/hep/ewmiller/anomaly-detection/processed_files/MC/ID",
    filenames=[
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run2air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run2water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run3air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run4air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run4water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run5water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run6air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run7water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run8air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run8water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run9water_v1.root'
    ],
    branches=["Pmu", "CosThetamu", "RecoPiMom", "RecoPiDirX", "RecoPiDirY", "RecoPiDirZ", "RecoProtonMom", "RecoProtonDirX", "RecoProtonDirY", "RecoProtonDirZ"],
    branch_scaling=np.array([0.2e-3, 1.0, 0.2e-3, 1.0, 1.0, 1.0, 0.2e-3, 1.0, 1.0, 1.0], dtype=np.float32),
    branch_mask_vals=np.array([-999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0]),
    branch_mask_replace_vals=np.array([0.0, -1.0, 0.0, -2.0, -2.0, -2.0, 0.0, -2.0, -2.0, -2.0], dtype=np.float32),
    filter="(isSRC!=1) & (RecoProton_Topo>0) & (RecoPi_Topo>0)" #"q0<2000.0"
)

ood_ds = InMemoryND280EventDataset(
    #root="/home/hep/ewmiller/anomaly-detection/processed_files/MC/OOD",
    filenames=[
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run2air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run2water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run3air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run4air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run4water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run5water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run6air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run7water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run8air_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run8water_v1.root',
        '/vols/t2k/nd280-OA2022/FDS-inputs/FDS_run9water_v1.root'
    ],
    branches=["Pmu", "CosThetamu", "RecoPiMom", "RecoPiDirX", "RecoPiDirY", "RecoPiDirZ", "RecoProtonMom", "RecoProtonDirX", "RecoProtonDirY", "RecoProtonDirZ"],
    branch_scaling=np.array([0.2e-3, 1.0, 0.2e-3, 1.0, 1.0, 1.0, 0.2e-3, 1.0, 1.0, 1.0], dtype=np.float32),
    branch_mask_vals=np.array([-999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0]),
    branch_mask_replace_vals=np.array([0.0, -1.0, 0.0, -2.0, -2.0, -2.0, 0.0, -2.0, -2.0, -2.0], dtype=np.float32),
    filter="(isSRC==1) & (RecoProton_Topo>0) & (RecoPi_Topo>0)" #"q0>= 2000.0"
)

train_ds.process()
ood_ds.process()


In [ ]:
print(f'Number of training examples: {len(train_ds)}')
print(f'Number of OOD examples:      {len(ood_ds)}')


raw_dist = np.ndarray((1, train_ds.get_n_features()))
ood_dist = np.ndarray((1, ood_ds.get_n_features()))

print()
print("##### Making ID and OOD raw distributions ######")

train_loader = DataLoader(train_ds, batch_size=4096, shuffle=True)
progressbar = tqdm(enumerate(train_loader), total=len(train_loader)-1)
for batch_n, (x, n) in progressbar:
    raw_dist = np.concatenate((raw_dist, x), axis=0)
    progressbar.update()

ood_loader = DataLoader(ood_ds, batch_size=4096, shuffle=True)
progressbar = tqdm(enumerate(ood_loader), total=len(ood_loader)-1)
for batch_n, (x, n) in progressbar:
    ood_dist = np.concatenate((ood_dist, x), axis=0)
    progressbar.update()


In [ ]:
import matplotlib.lines as mlines

def make_corner_plot(id_dist, ood_dist, ranges, titles, n_bins=20):
    figure = corner.corner(
        id_dist, 
        range  = ranges, 
        titles = titles,
        title_fmt = None,
        hist_kwargs={"color": 'tab:green', "alpha": 0.3, "fill": True, },
        color = "tab:green",
        weights = np.ones(raw_dist.shape[0]) / raw_dist.shape[0],
        show_titles = True,
        plot_contours = True,
        plot_datapoints = False,
        fill_contours=True,
        plot_density=False,
        density = True,
        hist2d_kwargs = {"no_fill_contours": True, "plot_datapoints": False},
        hist1d_kwargs = {"density": True},
        bins = n_bins,
        quiet = True,
    )

    corner.corner(
        ood_dist, 
        range  = ranges,
        titles = titles,
        title_fmt = None,
        hist_kwargs={"color": 'tab:orange', "alpha": 0.3, "fill": True, },
        color="tab:orange",
        weights = np.ones(ood_dist.shape[0]) / ood_dist.shape[0],
        show_titles = True,
        plot_contours = True,
        fill_contours = True,
        plot_datapoints = False,
        plot_density=False,
        density=True,
        hist2d_kwargs = {"no_fill_contours": True, "plot_datapoints": False, "new_fig": False},
        hist1d_kwargs = {"density": True},
        fig = figure,
        bins = n_bins, 
        quiet = True,
    )

    id_line = mlines.Line2D([], [], color="tab:green", label="In Distribution (ID)")
    ood_line = mlines.Line2D([], [], color="tab:orange", label='Out of Distribution (OOD)')

    figure.legend(handles=[id_line, ood_line], bbox_to_anchor=(0., 0.8, 0.9, .0), loc=4, fontsize=24)

    return figure

make_corner_plot(raw_dist, ood_dist, 
    ranges = [(-0.2, 1.5), (0.0,1.1),   (-0.2, 1.5), (-1.1,1.1),   (-1.1, 1.1),  (0.0, 1.0),   (-0.2, 1.5), (-1.1,1.1),  (-1.1, 1.1), (0.0, 1.1)],
    titles = ["p mu",      "ctheta mu", "p pi",      "pi dir [x]", "pi dir [y]", "pi dir [z]", "p p",       "p dir [x]", "p dir[y]",  "p dir[z]"],
).show()

In [ ]:
def get_latent_dists(model, data_loader, quiet=False):
    
    progressbar = tqdm(enumerate(data_loader), total=len(data_loader), desc = "loading data", leave=False, disable=quiet) 

    enc_dist = np.ndarray((1, n_features))
    llh_dist = np.ndarray((1, 1))

    if isinstance(model, nf.NormalizingFlowVAE):
        for batch_n, (x, n) in progressbar:
            x = x.to(device)
            encoded, log_q, log_p = model(x, num_samples)
            decoded = model.decoder(encoded)
            plt.scatter(x[:,0].detach(),decoded[:,0,0].detach(), c=[0.2, 0.4, 0.8, 0.3], s=0.2)


    if isinstance(model, VAE):
        for batch_n, (x, n) in progressbar:
            x = x.to(device)
            encoded = model.encode(x)
            plt.scatter(x[:, 1].detach(), encoded[:, 1].detach(), c=[0.2, 0.4, 0.8, 0.3], s=0.2)


    if isinstance(model, NICEModel):

        for batch_n, (x, n) in progressbar:
            x = x.to(device)
            encoded = model(x)
            enc_dist = np.concatenate((enc_dist, encoded.detach().to(torch.device("cpu"))), axis=0)
            llh_dist = np.concatenate((llh_dist, NICE_gaussian_loss(encoded.detach().to(torch.device("cpu")), model.scaling_diag.detach().to(torch.device("cpu")), keepdim=True)), axis=0)

    return enc_dist, llh_dist

In [ ]:
n_features = train_ds.get_n_features()

######################
## set up model ##
######################

# model = VAE(6, 2) 

# model = NFVAE(
#     n_bottleneck = 2,
#     hidden_units_encoder = [n_features, 8, 4, n_bottleneck*2],
#     hidden_units_decoder = [n_bottleneck, 4, 8, n_features],
#     n_flows = 0,
#     flow_type = "Planar",
#     device = device,
# )

model = NICEModel(n_features, 5, [64, 128, 256, 128, 64])

model.to(device)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

model.compile()
model.train()

print("##### Model #####'")
for module in model.modules():
    print(module)

print(f"trainable parameters: {params}")

In [ ]:
n_epochs = 300

train_loader:DataLoader = DataLoader(train_ds, batch_size=100000, shuffle=True)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

epoch_progressbar = tqdm(range(n_epochs), total = n_epochs, desc="epoch")
for epoch_n in epoch_progressbar: 
    
    epoch_loss = 0.0
    n_batches  = 0

    batch_progressbar = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for batch_n, (x, n) in batch_progressbar:

        x = x.to(device)
        optimizer.zero_grad()
        loss = model.train_batch(x)
        optimizer.step()
        
        batch_progressbar.set_description(f"loss: {loss.item():.4f}")

        # update running mean loss
        epoch_loss += loss.item()
        n_batches += 1

    
    # make plot of the latent space
    id_encoded,  id_llh  = get_latent_dists(model, train_loader, quiet=True)
    ood_encoded, ood_llh = get_latent_dists(model, ood_loader, quiet=True)

    fig = make_corner_plot(id_encoded, ood_encoded,
        ranges = [(-1.5, 1.5), (-1.5,1.5), (-1.5, 1.5), (-1.5,1.5), (-1.5, 1.5), (-1.5,1.5), (-1.5, 1.5), (-1.5,1.5), (-1.5, 1.5), (-1.5,1.5)],
        titles = None,
        n_bins = 40
    )

    fig.savefig(f"plots/latent_dist-epoch-{epoch_n:04}.png")
    fig.clf()
        
    epoch_loss /= n_batches    
    epoch_progressbar.set_description(f"epoch {epoch_n} loss: {epoch_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), "models/src_detector_5-layer-64_128_256_128_64.pt")

In [ ]:

id_encoded,  id_llh  = get_latent_dists(model, train_loader)
ood_encoded, ood_llh = get_latent_dists(model, ood_loader)

plt.hist([ood_llh[:,0], id_llh[:,0],], color = ["tab:orange", "tab:green"], bins=100, range=(-50, 5), histtype="step", label=["OOD", "ID"], fill=False)
plt.title("ID vs OOD LLH Distribution")
plt.yscale('log')
plt.legend()
plt.show()

plt.hist([ood_llh[:,0], id_llh[:,0]], color = ["tab:orange", "tab:green"], bins=50, range=(-50, 5), histtype='step', label=["OOD", "IID"], density=True, fill=False)
plt.title("ID vs OOD LLH Distribution - Normalised")
plt.xlabel("LLH")
plt.yscale('log')
plt.legend()
plt.show()

LLH_CUT = 5.0
feat_names = ["p mu", "ctheta mu", "p pi", "ctheta pi", "p p", "ctheta p", "reco N pi", "reco N Prot"]
feat_ranges = [(0.001, 1), (-0.999,1), (0.001, 1), (-0.999,1), (0.001, 1), (-0.999,1), (0, 1.0), (0, 1.0)]
far_outliers = ood_llh[:, 0] < LLH_CUT


In [ ]:

make_corner_plot(id_encoded, ood_encoded,
    ranges = [(-1.5, 1.5), (-1.5,1.5), (-1.5, 1.5), (-1.5,1.5), (-1.5, 1.5), (-1.5,1.5), (-1.5, 1.5), (-1.5,1.5), (-1.5, 1.5), (-1.5,1.5)],
    titles = None,
    n_bins = 40
).show()
